In [ ]:
import os
os.chdir('..')
os.environ['LINE_PROFILE'] = '1'

In [ ]:
from datetime import timezone
from pathlib import Path

import polars as pl
from dateutil import parser
from datetime import timedelta

from config import DataFrequency, BINANCE_DATA_DIR, TradeType

from util.ts_manager import TSManager
from aws.kline.parse import read_csv
from aws.client_async import AwsKlineClient

In [ ]:
symbol = 'BTCUSDT'
time_interval = '1m'
base_dir = AwsKlineClient.get_base_dir(TradeType.spot, data_freq=DataFrequency.daily)
aws_dir = BINANCE_DATA_DIR / 'aws_data' / base_dir / symbol / time_interval
aws_kline_files = sorted(aws_dir.glob(f'*.zip'))

# d = {f.stem: read_csv(f) for f in aws_kline_files}
# print(d.keys())

In [ ]:
mgr = TSManager('/home/lostleaf/crypto_data/binance_data/parsed_data/spot/klines/BTCUSDT/1m/')
c = mgr.get_row_count_per_date()
c.filter(pl.col('row_count') != 1440)

df1 = mgr.read_all()

print(df1.shape)

ldf1 = df1.lazy()

In [ ]:
api_dir = Path('/home/lostleaf/crypto_data/binance_data/api_data/kline/spot/BTCUSDT/1m')
api_files = sorted(api_dir.glob('*.pqt'))
ldf2 = pl.scan_parquet(api_files).drop('candle_end_time')

ldf = pl.concat([ldf1, ldf2])
ldf = ldf.unique('candle_begin_time', keep='last').sort('candle_begin_time')
ldf = ldf.with_columns(
    (pl.col('quote_volume') / pl.col('volume'))
    .fill_null(pl.col('open'))
    .clip(pl.col('low'), pl.col('high'))
    .alias('vwap1m')
)

df = ldf.collect()
df.tail(10)